# Knowledge Distillation for EfficientNetV2 on CIFAR-100 - Version 3.1 (Tuned DKD)

This notebook implements **State-of-the-Art** knowledge distillation with:

- **Decoupled Knowledge Distillation (DKD)** - Separates target and non-target class knowledge (Zhao et al., CVPR 2022)
- **AutoAugment + Random Erasing** - Advanced data augmentation
- **CutMix + Mixup** - Sample mixing strategies
- **Label Smoothing + LR Warmup** - Training stability

---

## Version History:

- **v1 (Baseline):** Standard KD + CutMix/Mixup → 72.34% accuracy
- **v2 (Enhanced):** + AutoAugment + Label Smoothing + LR Warmup → 74.20% accuracy
- **v3 (DKD):** + Decoupled Knowledge Distillation (β=8.0) → 72.69% (over-regularized)
- **v3.1 (Tuned DKD):** Reduced β from 8.0 to 2.0 → Target: 75%+ accuracy

---

## Key Changes in v3.1:

1. **Reduced DKD_BETA** from 8.0 to 2.0 (less aggressive NCKD weight)
2. **Reuse Teacher v3** (76.02% accuracy) - no retraining needed
3. **New student model name:** `student_v3_1_dkd_beta2`

**Diagnosis:** v3 suffered from over-regularization due to high β combined with strong augmentation (Mixup/CutMix).

---


In [1]:
# Cell 1: Setup and Imports
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import efficientnet_v2_s, efficientnet_v2_l, EfficientNet_V2_S_Weights, EfficientNet_V2_L_Weights
from tqdm import tqdm
import copy
import numpy as np
import matplotlib.pyplot as plt
import glob
import random
from pathlib import Path

# Setup Directories (Local paths)
PROJECT_ROOT = Path('./outputs')
MODEL_DIR = PROJECT_ROOT / 'models'
DATA_DIR = PROJECT_ROOT / 'data'
CHECKPOINT_DIR = PROJECT_ROOT / 'checkpoints'

MODEL_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Directories ready at: {PROJECT_ROOT.absolute()}")

# Reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.benchmark = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Directories ready at: d:\Projects\MasterProject\code\outputs
Using device: cuda
GPU: NVIDIA GeForce RTX 5070 Laptop GPU


In [2]:
# Cell 2: Experiment Configuration (Version 3.1 - Tuned DKD Beta)
# ==========================================
# HYPERPARAMETERS
# ==========================================
NUM_EPOCHS = 200
TEACHER_EPOCHS = 300        # Extended teacher training for v3
BATCH_SIZE = 128
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.05
PATIENCE = 30
WARMUP_EPOCHS = 5           # LR warmup

# Distillation Params (DKD - Tuned for v3.1)
DKD_ALPHA = 1.0             # Weight for Target Class Knowledge Distillation (TCKD)
DKD_BETA = 2.0              # REDUCED from 8.0 to 2.0 - Less aggressive NCKD
TEMPERATURE = 4.0           # Softmax Temperature
LABEL_SMOOTHING = 0.1       # Label smoothing for hard loss

# Augmentation Params
MIXUP_ALPHA = 0.8
CUTMIX_ALPHA = 1.0
CHECKPOINT_FREQUENCY = 20
NUM_CLASSES = 100

# Version 3.1 Model Names
TEACHER_NAME = "teacher_v3"              # Reuse the strong teacher (76.02%)
STUDENT_NAME = "student_v3_1_dkd_beta2"  # New student with tuned beta

print(f"{'='*60}")
print(f"VERSION 3.1 CONFIG - Tuned Beta Parameter")
print(f"{'='*60}")
print(f"Teacher: {TEACHER_NAME} (Accuracy: 76.02%)")
print(f"Training: Epochs={NUM_EPOCHS} | Batch={BATCH_SIZE} | LR={LEARNING_RATE}")
print(f"DKD: Alpha(TCKD)={DKD_ALPHA}, Beta(NCKD)={DKD_BETA} (REDUCED from 8.0)")
print(f"Augmentation: AutoAugment + RandomErasing + CutMix/Mixup")
print(f"Student Model: {STUDENT_NAME}")
print(f"{'='*60}")

VERSION 3.1 CONFIG - Tuned Beta Parameter
Teacher: teacher_v3 (Accuracy: 76.02%)
Training: Epochs=200 | Batch=128 | LR=0.001
DKD: Alpha(TCKD)=1.0, Beta(NCKD)=2.0 (REDUCED from 8.0)
Augmentation: AutoAugment + RandomErasing + CutMix/Mixup
Student Model: student_v3_1_dkd_beta2


In [3]:
# Cell 3: Data Loading (Enhanced with AutoAugment + RandomErasing)
from torchvision.transforms import autoaugment

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    autoaugment.AutoAugment(policy=autoaugment.AutoAugmentPolicy.CIFAR10),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
    transforms.RandomErasing(p=0.25, scale=(0.02, 0.2)),  # Cutout-like augmentation
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

trainset = torchvision.datasets.CIFAR100(root=str(DATA_DIR), train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, 
                                          num_workers=4, pin_memory=True, drop_last=True)

testset = torchvision.datasets.CIFAR100(root=str(DATA_DIR), train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, 
                                         num_workers=4, pin_memory=True)

print(f"Data loaded: {len(trainset)} Training, {len(testset)} Test images")
print(f"  Augmentation: AutoAugment + RandomErasing + CutMix/Mixup")

Data loaded: 50000 Training, 10000 Test images
  Augmentation: AutoAugment + RandomErasing + CutMix/Mixup


In [4]:
# Cell 4: Helper Functions (DKD Loss + Augmentations + Utilities)

# ==========================================
# 1. Decoupled Knowledge Distillation (DKD) Loss
# Based on: Zhao et al. "Decoupled Knowledge Distillation", CVPR 2022
# https://openaccess.thecvf.com/content/CVPR2022/papers/Zhao_Decoupled_Knowledge_Distillation_CVPR_2022_paper.pdf
# ==========================================

def _get_gt_mask(logits, target):
    """Get mask for target (ground truth) class"""
    target = target.reshape(-1)
    mask = torch.zeros_like(logits).scatter_(1, target.unsqueeze(1), 1).bool()
    return mask

def _get_other_mask(logits, target):
    """Get mask for non-target classes"""
    target = target.reshape(-1)
    mask = torch.ones_like(logits).scatter_(1, target.unsqueeze(1), 0).bool()
    return mask

def cat_mask(t, mask1, mask2):
    """Concatenate masked values"""
    t1 = (t * mask1).sum(dim=1, keepdims=True)
    t2 = (t * mask2).sum(1, keepdims=True)
    rt = torch.cat([t1, t2], dim=1)
    return rt

def dkd_loss(student_logits, teacher_logits, target, alpha=1.0, beta=8.0, temperature=4.0):
    """
    Decoupled Knowledge Distillation (DKD) Loss.
    
    Separates knowledge into:
    - TCKD (Target Class Knowledge Distillation): Knowledge about the correct class
    - NCKD (Non-Target Class Knowledge Distillation): Knowledge about class relationships
    
    Args:
        student_logits: Student model outputs
        teacher_logits: Teacher model outputs
        target: Ground truth labels
        alpha: Weight for TCKD (default: 1.0)
        beta: Weight for NCKD (default: 8.0, crucial for performance)
        temperature: Softmax temperature (default: 4.0)
    
    Returns:
        Combined DKD loss
    """
    # Get masks for target and non-target classes
    gt_mask = _get_gt_mask(student_logits, target)
    other_mask = _get_other_mask(student_logits, target)
    
    # Calculate probabilities with temperature scaling
    pred_student = F.softmax(student_logits / temperature, dim=1)
    pred_teacher = F.softmax(teacher_logits / temperature, dim=1)
    
    # Separate target and non-target probabilities
    pred_student_cat = cat_mask(pred_student, gt_mask, other_mask)
    pred_teacher_cat = cat_mask(pred_teacher, gt_mask, other_mask)
    
    # Target Class Knowledge Distillation (TCKD)
    log_pred_student = torch.log(pred_student_cat + 1e-8)
    tckd_loss = (
        F.kl_div(log_pred_student, pred_teacher_cat, reduction='batchmean')
        * (temperature ** 2)
    )
    
    # Non-Target Class Knowledge Distillation (NCKD)
    # Mask out target class with large negative value
    pred_teacher_part2 = F.softmax(
        teacher_logits / temperature - 1000.0 * gt_mask, dim=1
    )
    log_pred_student_part2 = F.log_softmax(
        student_logits / temperature - 1000.0 * gt_mask, dim=1
    )
    
    nckd_loss = (
        F.kl_div(log_pred_student_part2, pred_teacher_part2, reduction='batchmean')
        * (temperature ** 2)
    )
    
    # Combined DKD loss
    return alpha * tckd_loss + beta * nckd_loss

def dkd_loss_mixup(student_logits, teacher_logits, labels_a, labels_b, lam,
                   alpha=1.0, beta=8.0, temperature=4.0, label_smoothing=0.1):
    """
    DKD Loss for mixed samples (CutMix/Mixup compatible)
    Uses weighted combination of DKD losses for both label sets
    """
    # DKD for both label sets
    dkd_a = dkd_loss(student_logits, teacher_logits, labels_a, alpha, beta, temperature)
    dkd_b = dkd_loss(student_logits, teacher_logits, labels_b, alpha, beta, temperature)
    
    # Weighted DKD loss
    soft_loss = lam * dkd_a + (1 - lam) * dkd_b
    
    # Hard loss with label smoothing
    hard_loss = lam * F.cross_entropy(student_logits, labels_a, label_smoothing=label_smoothing) + \
                (1 - lam) * F.cross_entropy(student_logits, labels_b, label_smoothing=label_smoothing)
    
    # Combined loss (DKD already weighted by alpha/beta, so we just add hard loss)
    # Using 0.1 weight for hard loss to not overpower DKD
    loss = soft_loss + 0.1 * hard_loss
    
    # NaN safety check
    if torch.isnan(loss):
        return hard_loss
    
    return loss

# ==========================================
# 2. Standard KD Loss (kept for comparison)
# ==========================================
def kd_loss(student_logits, teacher_logits, labels, temp=4.0, alpha=0.7, label_smoothing=0.1):
    """Standard Knowledge Distillation Loss (for comparison)"""
    soft_student = F.log_softmax(student_logits / temp, dim=1)
    soft_teacher = F.softmax(teacher_logits / temp, dim=1)
    soft_loss = F.kl_div(soft_student, soft_teacher, reduction='batchmean') * (temp ** 2)
    hard_loss = F.cross_entropy(student_logits, labels, label_smoothing=label_smoothing)
    return alpha * soft_loss + (1 - alpha) * hard_loss

# ==========================================
# 3. Augmentations: Mixup & CutMix
# ==========================================
def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)
    cx = np.random.randint(W)
    cy = np.random.randint(H)
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

def cutmix_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)
    bbx1, bby1, bbx2, bby2 = rand_bbox(x.size(), lam)
    x[:, :, bbx1:bbx2, bby1:bby2] = x[index, :, bbx1:bbx2, bby1:bby2]
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (x.size()[-1] * x.size()[-2]))
    y_a, y_b = y, y[index]
    return x, y_a, y_b, lam

# ==========================================
# 4. Utilities (Save/Load/Evaluate)
# ==========================================
def evaluate_model_with_loss(model, dataloader, criterion):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    avg_loss = running_loss / len(dataloader)
    return acc, avg_loss

def save_checkpoint(model, optimizer, scheduler, epoch, best_acc, history, model_name, epochs_no_improve):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_acc': best_acc,
        'history': history,
        'epochs_no_improve': epochs_no_improve
    }
    path = CHECKPOINT_DIR / f"{model_name}_epoch{epoch+1}.pth"
    torch.save(checkpoint, path)
    print(f"  Checkpoint saved: {path}")
    
def load_checkpoint(model, optimizer, scheduler, model_name):
    checkpoints = sorted(glob.glob(str(CHECKPOINT_DIR / f"{model_name}_epoch*.pth")))
    if not checkpoints:
        return None
    latest = checkpoints[-1]
    print(f"  Loading checkpoint: {latest}")
    checkpoint = torch.load(latest, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    return checkpoint

def cleanup_old_checkpoints(model_name, keep=3):
    checkpoints = sorted(glob.glob(str(CHECKPOINT_DIR / f"{model_name}_epoch*.pth")))
    if len(checkpoints) > keep:
        for chk in checkpoints[:-keep]:
            os.remove(chk)
            print(f"  Cleaned up: {os.path.basename(chk)}")

print("Helper functions loaded:")
print("  - DKD Loss (Decoupled Knowledge Distillation)")
print("  - Standard KD Loss (for comparison)")
print("  - CutMix, Mixup augmentations")
print("  - Checkpoint utilities")

Helper functions loaded:
  - DKD Loss (Decoupled Knowledge Distillation)
  - Standard KD Loss (for comparison)
  - CutMix, Mixup augmentations
  - Checkpoint utilities


In [5]:
# Cell 5: Training Loop (Version 3 - with DKD support)
def train_model_v3(model, dataloader, optimizer, scheduler, num_epochs, model_name, 
                   teacher_model=None, use_dkd=True,
                   dkd_alpha=1.0, dkd_beta=8.0, temperature=4.0, label_smoothing=0.1,
                   patience=30, grad_clip=1.0, warmup_epochs=5):
    """
    Version 3 Training loop with:
    - Decoupled Knowledge Distillation (DKD) loss
    - CutMix/Mixup augmentation
    - Learning Rate Warmup
    - Mixed precision training
    - Gradient clipping
    - NaN detection and recovery
    """
    
    # 1. Load Checkpoint
    checkpoint = load_checkpoint(model, optimizer, scheduler, model_name)
    if checkpoint:
        start_epoch = checkpoint['epoch'] + 1
        best_acc = checkpoint['best_acc']
        history = checkpoint['history']
        epochs_no_improve = checkpoint['epochs_no_improve']
        best_model_wts = copy.deepcopy(model.state_dict())
        print(f"  Resuming from epoch {start_epoch}, Best Acc: {best_acc:.2f}%")
    else:
        start_epoch = 0
        best_acc = 0.0
        history = {'train_loss': [], 'val_loss': [], 'val_accuracy': []}
        epochs_no_improve = 0
        best_model_wts = copy.deepcopy(model.state_dict())
        print(f"  Starting fresh training...")

    # 2. Setup
    scaler = torch.amp.GradScaler('cuda', enabled=torch.cuda.is_available())
    val_criterion = nn.CrossEntropyLoss()
    
    # Store base LR for warmup
    base_lr = optimizer.param_groups[0]['lr']
    
    if teacher_model:
        teacher_model.eval()
        for param in teacher_model.parameters():
            param.requires_grad = False
        loss_type = "DKD" if use_dkd else "Standard KD"
        print(f"  Using {loss_type} loss with Teacher guidance")

    # 3. Training Loop
    for epoch in range(start_epoch, num_epochs):
        model.train()
        running_loss = 0.0
        valid_batches = 0
        
        # Learning Rate Warmup
        if epoch < warmup_epochs:
            warmup_lr = base_lr * (epoch + 1) / warmup_epochs
            for param_group in optimizer.param_groups:
                param_group['lr'] = warmup_lr
        
        loop = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
        
        for inputs, labels in loop:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Randomly choose Mixup (50%) or CutMix (50%)
            use_cutmix = np.random.rand() > 0.5
            if use_cutmix:
                inputs_aug, labels_a, labels_b, lam = cutmix_data(inputs.clone(), labels, alpha=CUTMIX_ALPHA)
            else:
                inputs_aug, labels_a, labels_b, lam = mixup_data(inputs, labels, alpha=MIXUP_ALPHA)
            
            optimizer.zero_grad()
            
            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                # Student Forward
                student_outputs = model(inputs_aug)
                
                # Loss Calculation
                if teacher_model:
                    # Teacher Forward (no grad)
                    with torch.no_grad():
                        teacher_outputs = teacher_model(inputs_aug)
                    
                    if use_dkd:
                        # DKD Loss (Version 3)
                        loss = dkd_loss_mixup(
                            student_outputs, teacher_outputs, 
                            labels_a, labels_b, lam,
                            alpha=dkd_alpha, beta=dkd_beta, 
                            temperature=temperature, label_smoothing=label_smoothing
                        )
                    else:
                        # Standard KD Loss (for comparison)
                        loss = lam * kd_loss(student_outputs, teacher_outputs, labels_a, temperature, 0.7, label_smoothing) + \
                               (1 - lam) * kd_loss(student_outputs, teacher_outputs, labels_b, temperature, 0.7, label_smoothing)
                else:
                    # Standard CE with Label Smoothing for Teacher training
                    loss = lam * F.cross_entropy(student_outputs, labels_a, label_smoothing=label_smoothing) + \
                           (1 - lam) * F.cross_entropy(student_outputs, labels_b, label_smoothing=label_smoothing)
            
            # Skip batch if loss is NaN
            if torch.isnan(loss):
                loop.set_postfix(loss="NaN-skip")
                continue
            
            # Backward
            scaler.scale(loss).backward()
            
            if grad_clip > 0:
                scaler.unscale_(optimizer)
                nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
                
            scaler.step(optimizer)
            scaler.update()
            
            running_loss += loss.item()
            valid_batches += 1
            loop.set_postfix(loss=f"{loss.item():.3f}")

        # Step Scheduler (only after warmup)
        if epoch >= warmup_epochs:
            scheduler.step()
        
        # Validation
        train_loss = running_loss / max(valid_batches, 1)
        val_acc, val_loss = evaluate_model_with_loss(model, testloader, val_criterion)
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_accuracy'].append(val_acc)
        
        current_lr = optimizer.param_groups[0]['lr']
        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | LR: {current_lr:.6f}")
        
        # Save Best
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), MODEL_DIR / f"{model_name}.pth")
            epochs_no_improve = 0
            print(f"  ★ New best model saved! Accuracy: {best_acc:.2f}%")
        else:
            epochs_no_improve += 1
            
        # Checkpointing
        if (epoch + 1) % CHECKPOINT_FREQUENCY == 0:
            save_checkpoint(model, optimizer, scheduler, epoch, best_acc, history, model_name, epochs_no_improve)
            cleanup_old_checkpoints(model_name)
            
        # Early Stopping
        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            
    print(f"\n{'='*50}")
    print(f"Training complete. Best accuracy: {best_acc:.2f}%")
    print(f"{'='*50}")
    model.load_state_dict(best_model_wts)
    return model, history

print("Training function loaded (Version 3: DKD + LR Warmup + Label Smoothing)")

Training function loaded (Version 3: DKD + LR Warmup + Label Smoothing)


In [6]:
# Cell 6: Initialize Models
print("Loading Teacher (EfficientNetV2-L)...")
teacher_model = efficientnet_v2_l(weights=EfficientNet_V2_L_Weights.IMAGENET1K_V1)
teacher_model.classifier[1] = nn.Linear(teacher_model.classifier[1].in_features, NUM_CLASSES)
teacher_model = teacher_model.to(device)

print("Loading Student (EfficientNetV2-S)...")
student_model = efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1)
student_model.classifier[1] = nn.Linear(student_model.classifier[1].in_features, NUM_CLASSES)
student_model = student_model.to(device)

print("Models loaded")

Loading Teacher (EfficientNetV2-L)...
Loading Student (EfficientNetV2-S)...
Models loaded


In [7]:
# Cell 7: Train/Load Teacher Model (Version 3)
print("\n" + "="*70)
print("TEACHER MODEL (Version 3)")
print("="*70)

teacher_path = MODEL_DIR / f"{TEACHER_NAME}.pth"

# Also check for existing v1/v2 teacher to use as starting point
existing_teacher = MODEL_DIR / "teacher_model.pth"

if teacher_path.exists():
    print(f"Found existing Teacher v3: {teacher_path}")
    teacher_model.load_state_dict(torch.load(teacher_path, map_location=device))
elif existing_teacher.exists():
    print(f"Found existing Teacher (v1/v2): {existing_teacher}")
    print(f"Loading and continuing training for {TEACHER_EPOCHS} epochs...")
    teacher_model.load_state_dict(torch.load(existing_teacher, map_location=device))
    
    # Continue training with extended epochs
    opt_t = optim.AdamW(teacher_model.parameters(), lr=LEARNING_RATE * 0.1, weight_decay=WEIGHT_DECAY)  # Lower LR for fine-tuning
    sch_t = optim.lr_scheduler.CosineAnnealingLR(opt_t, T_max=100)  # Additional 100 epochs
    
    teacher_model, teacher_history = train_model_v3(
        teacher_model, trainloader, opt_t, sch_t, 
        num_epochs=100,  # Additional epochs
        model_name=TEACHER_NAME, 
        teacher_model=None,
        warmup_epochs=0  # No warmup for fine-tuning
    )
else:
    print(f"Training Teacher Model v3 from scratch ({TEACHER_EPOCHS} epochs)...")
    print("This may take a while...")
    
    # Re-initialize teacher
    teacher_model = efficientnet_v2_l(weights=EfficientNet_V2_L_Weights.IMAGENET1K_V1)
    teacher_model.classifier[1] = nn.Linear(teacher_model.classifier[1].in_features, NUM_CLASSES)
    teacher_model = teacher_model.to(device)
    
    opt_t = optim.AdamW(teacher_model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    sch_t = optim.lr_scheduler.CosineAnnealingLR(opt_t, T_max=TEACHER_EPOCHS - WARMUP_EPOCHS)
    
    teacher_model, teacher_history = train_model_v3(
        teacher_model, trainloader, opt_t, sch_t, 
        num_epochs=TEACHER_EPOCHS, 
        model_name=TEACHER_NAME, 
        teacher_model=None,
        warmup_epochs=WARMUP_EPOCHS
    )

# Evaluate Teacher
teacher_model.eval()
teacher_accuracy, _ = evaluate_model_with_loss(teacher_model, testloader, nn.CrossEntropyLoss())
print(f"\nTeacher v3 Accuracy: {teacher_accuracy:.2f}%")


TEACHER MODEL (Version 3)
Found existing Teacher v3: outputs\models\teacher_v3.pth

Teacher v3 Accuracy: 76.02%


In [8]:
# Cell 8: Train Student with DKD (Version 3.1 - Tuned Beta)
print("\n" + "="*70)
print("STUDENT MODEL with DKD (Version 3.1 - Tuned Beta)")
print("="*70)

student_path = MODEL_DIR / f"{STUDENT_NAME}.pth"

if student_path.exists():
    print(f"Found existing Student v3.1: {student_path}")
    student_model.load_state_dict(torch.load(student_path, map_location=device))
    distilled_accuracy, _ = evaluate_model_with_loss(student_model, testloader, nn.CrossEntropyLoss())
    print(f"Student v3.1 (DKD β=2.0) Accuracy: {distilled_accuracy:.2f}%")
else:
    print(f"\nStarting Decoupled Knowledge Distillation (DKD) v3.1...")
    print(f"  Key Change: DKD Beta REDUCED from 8.0 to {DKD_BETA}")
    print(f"  DKD Alpha (TCKD weight): {DKD_ALPHA}")
    print(f"  DKD Beta (NCKD weight): {DKD_BETA} ← REDUCED to fix over-regularization")
    print(f"  Temperature: {TEMPERATURE}")
    print(f"  Label Smoothing: {LABEL_SMOOTHING}")
    print(f"  LR Warmup: {WARMUP_EPOCHS} epochs")
    print(f"  Augmentation: AutoAugment + RandomErasing + CutMix/Mixup")
    
    # Re-initialize student model (fresh weights from ImageNet)
    student_model = efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1)
    student_model.classifier[1] = nn.Linear(student_model.classifier[1].in_features, NUM_CLASSES)
    student_model = student_model.to(device)
    
    # Optimizer & Scheduler
    opt_s = optim.AdamW(student_model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    sch_s = optim.lr_scheduler.CosineAnnealingLR(opt_s, T_max=NUM_EPOCHS - WARMUP_EPOCHS)
    
    # Run DKD Training with tuned beta
    trained_student, distilled_history = train_model_v3(
        model=student_model,
        dataloader=trainloader,
        optimizer=opt_s,
        scheduler=sch_s,
        num_epochs=NUM_EPOCHS,
        model_name=STUDENT_NAME,
        teacher_model=teacher_model,
        use_dkd=True,  # Enable DKD loss
        dkd_alpha=DKD_ALPHA,
        dkd_beta=DKD_BETA,  # Now 2.0 instead of 8.0
        temperature=TEMPERATURE,
        label_smoothing=LABEL_SMOOTHING,
        patience=PATIENCE,
        warmup_epochs=WARMUP_EPOCHS
    )
    
    distilled_accuracy, _ = evaluate_model_with_loss(trained_student, testloader, nn.CrossEntropyLoss())
    print(f"\nStudent v3.1 (DKD β=2.0) Final Accuracy: {distilled_accuracy:.2f}%")


STUDENT MODEL with DKD (Version 3.1 - Tuned Beta)

Starting Decoupled Knowledge Distillation (DKD) v3.1...
  Key Change: DKD Beta REDUCED from 8.0 to 2.0
  DKD Alpha (TCKD weight): 1.0
  DKD Beta (NCKD weight): 2.0 ← REDUCED to fix over-regularization
  Temperature: 4.0
  Label Smoothing: 0.1
  LR Warmup: 5 epochs
  Augmentation: AutoAugment + RandomErasing + CutMix/Mixup
  Starting fresh training...
  Using DKD loss with Teacher guidance


Epoch 1/200: 100%|██████████| 390/390 [01:18<00:00,  4.99it/s, loss=1.317]


Epoch 1/200 | Train Loss: 2.0525 | Val Loss: 3.4177 | Val Acc: 25.66% | LR: 0.000200
  ★ New best model saved! Accuracy: 25.66%


Epoch 2/200: 100%|██████████| 390/390 [01:18<00:00,  4.94it/s, loss=1.340]


Epoch 2/200 | Train Loss: 1.8003 | Val Loss: 2.7542 | Val Acc: 39.99% | LR: 0.000400
  ★ New best model saved! Accuracy: 39.99%


...

Epoch 198/200 | Train Loss: 1.0429 | Val Loss: 1.4996 | Val Acc: 73.04% | LR: 0.000000


Epoch 199/200: 100%|██████████| 390/390 [01:13<00:00,  5.27it/s, loss=0.502]


Epoch 199/200 | Train Loss: 1.0405 | Val Loss: 1.2395 | Val Acc: 73.95% | LR: 0.000000


Epoch 200/200: 100%|██████████| 390/390 [01:14<00:00,  5.26it/s, loss=2.992]


Epoch 200/200 | Train Loss: 1.0202 | Val Loss: 1.3270 | Val Acc: 73.83% | LR: 0.000000
  Checkpoint saved: outputs\checkpoints\student_v3_1_dkd_beta2_epoch200.pth
  Cleaned up: student_v3_1_dkd_beta2_epoch200.pth

Training complete. Best accuracy: 73.98%

Student v3.1 (DKD β=2.0) Final Accuracy: 73.98%


In [9]:
# Cell 9: Results Summary (Version 3.1)
print("\n" + "="*80)
print("VERSION 3.1 RESULTS SUMMARY - Tuned DKD (Beta=2.0)")
print("="*80)

# Evaluate all available models for comparison
criterion = nn.CrossEntropyLoss()

print("\n┌─────────────────────────────────────────────────────────────────────┐")
print("│                    MODEL COMPARISON TABLE                           │")
print("├──────────────────────────────┬──────────────┬───────────────────────┤")
print("│ Model                        │ Accuracy (%) │ Notes                 │")
print("├──────────────────────────────┼──────────────┼───────────────────────┤")

# Teacher v3
try:
    print(f"│ Teacher v3 (EfficientNet-L)  │ {teacher_accuracy:12.2f} │ Extended training     │")
except NameError:
    print(f"│ Teacher v3 (EfficientNet-L)  │ {'N/A':>12} │                       │")

# Student v3.1 (DKD with tuned beta)
try:
    print(f"│ Student v3.1 DKD β=2.0       │ {distilled_accuracy:12.2f} │ Tuned NCKD weight     │")
except NameError:
    print(f"│ Student v3.1 DKD β=2.0       │ {'N/A':>12} │                       │")

print("├──────────────────────────────┴──────────────┴───────────────────────┤")
print("│                    PREVIOUS VERSIONS (for comparison)              │")
print("├──────────────────────────────┬──────────────┬───────────────────────┤")
print("│ Student v1 (Baseline)        │        72.34 │ Standard KD           │")
print("│ Student v2 (Enhanced)        │        74.20 │ + AutoAug + Warmup    │")
print("│ Student v3 (DKD β=8.0)       │        72.69 │ Over-regularized      │")
print("└──────────────────────────────┴──────────────┴───────────────────────┘")

print("\n" + "="*80)
print("VERSION 3.1 ANALYSIS:")
try:
    gap = distilled_accuracy - teacher_accuracy
    retention = (distilled_accuracy / teacher_accuracy) * 100
    print(f"  Teacher v3 Accuracy:       {teacher_accuracy:.2f}%")
    print(f"  Student v3.1 Accuracy:     {distilled_accuracy:.2f}%")
    print(f"  Gap from Teacher:          {gap:+.2f}%")
    print(f"  Teacher Retention:         {retention:.1f}%")
    
    # Compare with v3
    v3_accuracy = 72.69
    improvement = distilled_accuracy - v3_accuracy
    print(f"\n  Improvement over v3 (β=8.0): {improvement:+.2f}%")
except NameError:
    print("  Run training cells first to see results.")
print("="*80)

print("\nVersion 3.1 Key Change:")
print(f"  ✓ DKD Beta REDUCED: 8.0 → {DKD_BETA}")
print(f"  ✓ Hypothesis: Less aggressive NCKD reduces over-regularization")
print(f"  ✓ Expected: Better balance with Mixup/CutMix augmentation")

print(f"\nModel Files:")
print(f"  Teacher v3:   {MODEL_DIR / TEACHER_NAME}.pth")
print(f"  Student v3.1: {MODEL_DIR / STUDENT_NAME}.pth")
print(f"\nCheckpoints: {CHECKPOINT_DIR.absolute()}")


VERSION 3.1 RESULTS SUMMARY - Tuned DKD (Beta=2.0)

┌─────────────────────────────────────────────────────────────────────┐
│                    MODEL COMPARISON TABLE                           │
├──────────────────────────────┬──────────────┬───────────────────────┤
│ Model                        │ Accuracy (%) │ Notes                 │
├──────────────────────────────┼──────────────┼───────────────────────┤
│ Teacher v3 (EfficientNet-L)  │        76.02 │ Extended training     │
│ Student v3.1 DKD β=2.0       │        73.98 │ Tuned NCKD weight     │
├──────────────────────────────┴──────────────┴───────────────────────┤
│                    PREVIOUS VERSIONS (for comparison)              │
├──────────────────────────────┬──────────────┬───────────────────────┤
│ Student v1 (Baseline)        │        72.34 │ Standard KD           │
│ Student v2 (Enhanced)        │        74.20 │ + AutoAug + Warmup    │
│ Student v3 (DKD β=8.0)       │        72.69 │ Over-regularized      │
└───────────